## ●Notebookの内容

準備後データ(コード2の「データ準備2」で作成したもの)の読み込み、指標の整理、欠損の確認、欠損の補完とその確認

# 1. パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)

# 2. 整理後データの読み込み

In [2]:
pickle_in = open("./organized_data2.pickle","rb") #/Users/nagashimahikaru/document
data_3_0 = pickle.load(pickle_in)
pickle_in.close()
display(data_3_0)

,銘柄名,期日,銘柄コード,ISINｺｰﾄﾞ,始値,高値,安値,終値,売買代金,調整済始値,...,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）
0,極洋,1994/12,1301,NaN,378,419,371,402,0,3780,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,45539364000,4020
1,極洋,1995/01,1301,NaN,407,413,358,386,0,4070,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,43726852000,3860
2,極洋,1995/02,1301,NaN,388,400,346,352,0,3880,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,39875264000,3520
3,極洋,1995/03,1301,NaN,352,380,335,340,0,3520,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,38515880000,3400
4,極洋,1995/04,1301,NaN,330,370,315,355,0,3300,...,NaN,NaN,0.946636,35.878976,74197.722,421,47691.722,-14.081633,40215110000,3550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997,JP3835650007,883,900,773,822,6024130900,883,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,79934568000,822
370821,ベルーナ,2021/09,9997,JP3835650007,822,892,819,854,6967618600,822,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,83046376000,854
370822,ベルーナ,2021/10,9997,JP3835650007,848,851,792,798,3203755900,848,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,77600712000,798
370823,ベルーナ,2021/11,9997,JP3835650007,793,795,665,667,4021996800,793,...,1.584172,1.541385,1.483134,8.300202,169274.32,1298,126223.32,169.854466,64861748000,667


In [3]:
#β値の2乗を作成し、結合
data_3_0_1 = pd.Series(data_3_0['対TOPIX β値(60ヵ月)']*data_3_0['対TOPIX β値(60ヵ月)'], name = '対TOPIX β値(60ヵ月) 2乗')
data_3 = pd.concat([data_3_0,data_3_0_1], axis = 1)

In [4]:
day_num_3 = data_3.期日.nunique(dropna = True) #期日の数(月数)
com_num_3 = data_3.shape[0]/day_num_3 #上場廃止や途中上場がない使用可能な企業数
#com_num_3 = data_3.銘柄コード.nunique(dropna = True) #上場廃止や途中上場がない使用可能な企業数
print('使用可能な企業数は', com_num_3,'、', '期日数は', day_num_3 ,'である。')

使用可能な企業数は 1141.0 、 期日数は 325 である。


# 3. 指標の整理

## 3.1 全指標の確認

In [5]:
data_3_1 = pd.DataFrame(data_3.columns).T
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(data_3_1)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192
0,銘柄名,期日,銘柄コード,ISINｺｰﾄﾞ,始値,高値,安値,終値,売買代金,調整済始値,調整済高値,調整済安値,調整済終値,配当込基準値(基準=100),東証業種名,東証業種ｺｰﾄﾞ,日経平均採用ﾌﾗｸﾞ,発行済株式数,売買単位,時価総額(上場株式数),TOPIX浮動株比率,対TOPIXｳｴｲﾄ,株価(終値)移動平均5日乖離率,株価(終値)移動平均6日乖離率,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均100日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),PBR(普通株式数･実績･連結優先),PSR(普通株式数･日経予想･連結優先),EPS(普通株式数･日経予想･連結優先),EV/EBITDA(連結優先),BP(普通株式数･実績･連結優先),配当利回り実績値,配当利回り予想値,実績PER(NEEDS算出),予想PER(NEEDS算出),実績PSR(NEEDS算出),予想PSR(NEEDS算出),実績PBR(NEEDS算出),ﾎﾞﾘｭｰﾑﾚｼｵ,出来高回転率(250日),過去1ヶ月ﾘﾀｰﾝ,過去3ヶ月ﾘﾀｰﾝ,過去6ヶ月ﾘﾀｰﾝ,過去1年ﾘﾀｰﾝ,将来1ヶ月ﾘﾀｰﾝ,将来3ヶ月ﾘﾀｰﾝ,将来6ヶ月ﾘﾀｰﾝ,将来1年ﾘﾀｰﾝ,配当込み収益率(月次),HV（3ヵ月）,HV（6ヵ月）,HV（1年）,対TOPIX β値(36ヵ月),対TOPIX β値(60ヵ月),対TOPIX α値(36ヵ月),対TOPIX α値(60ヵ月),資産合計,流動資産,固定資産,負債合計,流動負債,固定負債,純資産,株主資本,研究開発費,営業利益(年換算),販売費及び一般管理費(年換算),経常利益(年換算),当期利益(年換算),EBITDA(年換算),減価償却実施額(年換算),営業活動によるｷｬｯｼｭﾌﾛｰ(年換算),投資活動によるｷｬｯｼｭﾌﾛｰ(年換算),財務活動によるｷｬｯｼｭﾌﾛｰ(年換算),ｷｬｯｼｭﾌﾛｰ(年換算),1株当たり売上高,1株当たり経常利益,1株当たり利益,1株当たり利益(企業発表),1株当たり当期利益(潜在株式調整後),潜在株式調整後1株当たり利益_遡及調整後,1株当たり当期純利益_遡及調整後,1株当たりEBITDA,1株当たり自己資本(Astra算出),1株当たり純資産(企業発表),1株当たり純資産_遡及調整後,1株当たりｷｬｯｼｭﾌﾛｰ,1株当たり営業ｷｬｯｼｭﾌﾛｰ,流動比率,当座比率,固定比率,固定長期適合率,自己資本比率,負債比率,他人資本ｺｽﾄ(有利子負債金利),借入金(有利子負債)依存度,有利子負債比率(ﾃﾞｯﾄｴｸｲﾃｨﾚｼｵ),長期借入金比率,長短借入金比率,有形固定資産比率,自己資本対資本金比率,手元流動性比率(月),ｲﾝﾀﾚｽﾄｶﾊﾞﾚｯｼﾞﾚｼｵ,財務ﾚﾊﾞﾚｯｼﾞ,売上債権対買入債務比率,自己資本配当率,経常収支比率,自己金融比率,海外売上高比率,売上高売上総利益率,売上高営業利益率,売上高経常利益率,売上高当期利益率,売上高事業利益率,売上高利払後事業利益率,EBITDAﾏｰｼﾞﾝ,売上高原価率,売上高販管費率,売上高減価償却費率,売上高研究開発費比率,売上高純金利負担率,売上高支払利息割引料率,総資産営業利益率,総資産経常利益率,総資産事業利益率(ROA),総資産利払後事業利益率,総資産利益率,自己資本経常利益率,自己資本利益率(ROE),資本金収益率,企業利潤率,使用資本利益率(ROCE),特別利益影響度,税率,総資産回転率,流動資産回転率,固定資産回転率,有形固定資産回転率,棚卸資産回転率,売上債権回転率,買入債務回転率,棚卸資産回転日数,売上債権回転日数,買入債務回転日数,稼動有形固定資産経常利益率,稼動有形固定資産純利益率,償却前利益対当期利益率,自己資本伸び率,総資産伸び率,売上高伸び率,営業利益伸び率,経常利益伸び率,当期利益伸び率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,配当性向(企業発表),総還元性向,純資産配当率,純資産減少割合,年間配当総額,ｷｬｯｼｭﾌﾛｰ対流動負債比率,ｷｬｯｼｭﾌﾛｰ対有利子負債,ｷｬｯｼｭﾌﾛｰ対支払利息額比率,ｷｬｯｼｭﾌﾛｰ対負債比率,ｷｬｯｼｭﾌﾛｰ対長期負債残高比率,ｷｬｯｼｭﾌﾛｰ対営業利益比率,ｷｬｯｼｭﾌﾛｰ対当期利益比率,営業ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,営業ｷｬｯｼｭﾌﾛｰ対流動負債比率,営業ｷｬｯｼｭﾌﾛｰ対有利子負債,営業ｷｬｯｼｭﾌﾛｰ対負債比率,営業ｷｬｯｼｭﾌﾛｰ対長期負債残高比率,営業ｷｬｯｼｭﾌﾛｰ対自己資本比率,営業ｷｬｯｼｭﾌﾛｰ対設備投資比率(A),営業ｷｬｯｼｭﾌﾛｰ対設備投資比率(B),営業ｷｬｯｼｭﾌﾛｰ対営業利益比率,営業ｷｬｯｼｭﾌﾛｰ対当期利益比率,営業ｷｬｯｼｭﾌﾛｰ対投資ｷｬｯｼｭﾌﾛｰ比率,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,期末EV,期末株式時価,期末時価総額(普通株ﾍﾞｰｽ),時価総額前年比,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗


## 3.2 不要な指標の削除

'東証業種名','東証業種ｺｰﾄﾞ','ISINｺｰﾄﾞ'→普通にいらない. 

'時価総額(上場株式数)'→発行株数ベースにする. 

'株価(終値)移動平均5,6,25日乖離率'→短いのはいらない.

'始値', '高値', '安値', '終値'→調整済を採用. 

'調整済終値'→現値を採用. 

'対TOPIXｳｴｲﾄ'→時価総額データと被る. 

'配当利回り実績値'→予想値を用いる.

'実績PER(NEEDS算出)', '予想PER(NEEDS算出)' ,'実績PSR(NEEDS算出)', '予想PSR(NEEDS算出)', '実績PBR(NEEDS算出)' → 重複してしまう.

'純資産' → 後々自分で計算する.

将来リターン系→普通にいらない. 

In [6]:
#不要な指標を削除
data_3_2 = data_3.drop(['東証業種名','東証業種ｺｰﾄﾞ','ISINｺｰﾄﾞ','時価総額(上場株式数)','株価(終値)移動平均5日乖離率', '株価(終値)移動平均6日乖離率',
                          '始値', '高値', '安値', '終値', '調整済終値', '対TOPIXｳｴｲﾄ', '配当利回り実績値', 
                          '実績PER(NEEDS算出)', '予想PER(NEEDS算出)' ,'実績PSR(NEEDS算出)', '予想PSR(NEEDS算出)', '実績PBR(NEEDS算出)',
                          '純資産',
                          '1株当たり売上高', '1株当たり経常利益', '1株当たり利益', '1株当たり自己資本(Astra算出)', '1株当たり利益(企業発表)', 
                          '1株当たり当期利益(潜在株式調整後)', '潜在株式調整後1株当たり利益_遡及調整後','1株当たり当期純利益_遡及調整後',
                          '1株当たりEBITDA', '1株当たり純資産(企業発表)', '1株当たり純資産_遡及調整後', '1株当たりｷｬｯｼｭﾌﾛｰ', 
                          '1株当たり営業ｷｬｯｼｭﾌﾛｰ', '将来1ヶ月ﾘﾀｰﾝ', '将来3ヶ月ﾘﾀｰﾝ', '将来6ヶ月ﾘﾀｰﾝ', '将来1年ﾘﾀｰﾝ',
                          '売買代金', '売買単位', 'TOPIX浮動株比率', '発行済株式数', '日経平均採用ﾌﾗｸﾞ', '資産合計', '流動資産', '固定資産', '負債合計',
                          '流動負債', '固定負債', '株主資本', '研究開発費', '営業利益(年換算)', '販売費及び一般管理費(年換算)', '経常利益(年換算)',
                          '当期利益(年換算)', '期末時価総額(普通株ﾍﾞｰｽ)', '減価償却実施額(年換算)', '営業活動によるｷｬｯｼｭﾌﾛｰ(年換算)',
                          '投資活動によるｷｬｯｼｭﾌﾛｰ(年換算)', '財務活動によるｷｬｯｼｭﾌﾛｰ(年換算)', '時価総額前年比', '期末EV', 'EV/EBITDA(連結優先)',
                          '売上高研究開発費比率',
                          '配当性向(企業発表)', '純資産配当率', '純資産減少割合', #この行から下のやつはクロセクの中央値置き換えでも欠損あったもの
                          '営業ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ', '営業ｷｬｯｼｭﾌﾛｰ対流動負債比率', '営業ｷｬｯｼｭﾌﾛｰ対有利子負債', '営業ｷｬｯｼｭﾌﾛｰ対負債比率',
                          '営業ｷｬｯｼｭﾌﾛｰ対長期負債残高比率', '営業ｷｬｯｼｭﾌﾛｰ対自己資本比率', '営業ｷｬｯｼｭﾌﾛｰ対設備投資比率(A)', 
                          '営業ｷｬｯｼｭﾌﾛｰ対設備投資比率(B)', '営業ｷｬｯｼｭﾌﾛｰ対営業利益比率', '営業ｷｬｯｼｭﾌﾛｰ対当期利益比率',
                          '営業ｷｬｯｼｭﾌﾛｰ対投資ｷｬｯｼｭﾌﾛｰ比率', #ここまでがクロセクの中央値置き換えでも欠損あったもの
                          '配当込基準値(基準=100)', '株価(終値)移動平均100日乖離率', 'BP(普通株式数･実績･連結優先)', #この行から下のやつは先生と相談して決めたやつ
                          '過去6ヶ月ﾘﾀｰﾝ', 'HV（6ヵ月）', '対TOPIX β値(36ヵ月)', '対TOPIX α値(36ヵ月)', 'EBITDA(年換算)', 'ｷｬｯｼｭﾌﾛｰ(年換算)',
                          '借入金(有利子負債)依存度', '自己資本対資本金比率', '資本金収益率', '特別利益影響度', '棚卸資産回転日数',
                          '売上債権回転日数', '買入債務回転日数', '稼動有形固定資産経常利益率', '稼動有形固定資産純利益率', 
                          '償却前利益対当期利益率', '年間配当総額', 'ｷｬｯｼｭﾌﾛｰ対流動負債比率', 'ｷｬｯｼｭﾌﾛｰ対有利子負債', 'ｷｬｯｼｭﾌﾛｰ対支払利息額比率',
                          'ｷｬｯｼｭﾌﾛｰ対負債比率', 'ｷｬｯｼｭﾌﾛｰ対長期負債残高比率', 'ｷｬｯｼｭﾌﾛｰ対営業利益比率', 'ｷｬｯｼｭﾌﾛｰ対当期利益比率', '期末株式時価'] , axis=1) 

data_3_3 = pd.DataFrame(data_3_2.columns).T
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(data_3_3)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88
0,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),PBR(普通株式数･実績･連結優先),PSR(普通株式数･日経予想･連結優先),EPS(普通株式数･日経予想･連結優先),配当利回り予想値,ﾎﾞﾘｭｰﾑﾚｼｵ,出来高回転率(250日),過去1ヶ月ﾘﾀｰﾝ,過去3ヶ月ﾘﾀｰﾝ,過去1年ﾘﾀｰﾝ,配当込み収益率(月次),HV（3ヵ月）,HV（1年）,対TOPIX β値(60ヵ月),対TOPIX α値(60ヵ月),流動比率,当座比率,固定比率,固定長期適合率,自己資本比率,負債比率,他人資本ｺｽﾄ(有利子負債金利),有利子負債比率(ﾃﾞｯﾄｴｸｲﾃｨﾚｼｵ),長期借入金比率,長短借入金比率,有形固定資産比率,手元流動性比率(月),ｲﾝﾀﾚｽﾄｶﾊﾞﾚｯｼﾞﾚｼｵ,財務ﾚﾊﾞﾚｯｼﾞ,売上債権対買入債務比率,自己資本配当率,経常収支比率,自己金融比率,海外売上高比率,売上高売上総利益率,売上高営業利益率,売上高経常利益率,売上高当期利益率,売上高事業利益率,売上高利払後事業利益率,EBITDAﾏｰｼﾞﾝ,売上高原価率,売上高販管費率,売上高減価償却費率,売上高純金利負担率,売上高支払利息割引料率,総資産営業利益率,総資産経常利益率,総資産事業利益率(ROA),総資産利払後事業利益率,総資産利益率,自己資本経常利益率,自己資本利益率(ROE),企業利潤率,使用資本利益率(ROCE),税率,総資産回転率,流動資産回転率,固定資産回転率,有形固定資産回転率,棚卸資産回転率,売上債権回転率,買入債務回転率,自己資本伸び率,総資産伸び率,売上高伸び率,営業利益伸び率,経常利益伸び率,当期利益伸び率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗


# 4. 欠損の確認

In [7]:
nan_num_1 = data_3_2.isnull().sum() #欠損の数を指標ごとに算出
nan_num_2 = pd.DataFrame(nan_num_1) #Seriesをデータフレーム化
nan_num_3 = nan_num_2[nan_num_2[0] != 0]  #欠損がある指標を表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(nan_num_3.T)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
#nan_num_3.index.values.tolist() #欠損がある指標を表示

,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),PBR(普通株式数･実績･連結優先),PSR(普通株式数･日経予想･連結優先),EPS(普通株式数･日経予想･連結優先),配当利回り予想値,ﾎﾞﾘｭｰﾑﾚｼｵ,出来高回転率(250日),過去1ヶ月ﾘﾀｰﾝ,過去3ヶ月ﾘﾀｰﾝ,過去1年ﾘﾀｰﾝ,配当込み収益率(月次),HV（3ヵ月）,HV（1年）,対TOPIX β値(60ヵ月),対TOPIX α値(60ヵ月),流動比率,当座比率,固定比率,固定長期適合率,自己資本比率,負債比率,他人資本ｺｽﾄ(有利子負債金利),有利子負債比率(ﾃﾞｯﾄｴｸｲﾃｨﾚｼｵ),長期借入金比率,長短借入金比率,有形固定資産比率,手元流動性比率(月),ｲﾝﾀﾚｽﾄｶﾊﾞﾚｯｼﾞﾚｼｵ,財務ﾚﾊﾞﾚｯｼﾞ,売上債権対買入債務比率,自己資本配当率,経常収支比率,自己金融比率,海外売上高比率,売上高売上総利益率,売上高営業利益率,売上高経常利益率,売上高当期利益率,売上高事業利益率,売上高利払後事業利益率,EBITDAﾏｰｼﾞﾝ,売上高原価率,売上高販管費率,売上高減価償却費率,売上高純金利負担率,売上高支払利息割引料率,総資産営業利益率,総資産経常利益率,総資産事業利益率(ROA),総資産利払後事業利益率,総資産利益率,自己資本経常利益率,自己資本利益率(ROE),企業利潤率,使用資本利益率(ROCE),税率,総資産回転率,流動資産回転率,固定資産回転率,有形固定資産回転率,棚卸資産回転率,売上債権回転率,買入債務回転率,自己資本伸び率,総資産伸び率,売上高伸び率,営業利益伸び率,経常利益伸び率,当期利益伸び率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,対TOPIX β値(60ヵ月) 2乗
0,13,13,13,1017,1424,2680,3668,484,1158,3668,3516,1015,900,6,33,401,6,34,403,12756,12756,14407,17332,14929,14506,36,558,43599,31710,74623,42306,558,22683,25999,558,19920,39292,33754,41143,193116,20621,6377,36,36,9302,17982,20839,20645,2979,6403,28226,28226,11676,5435,23257,23257,5435,5928,5928,22683,11684,51064,5435,19782,19782,5435,25884,24583,24290,5435,5435,7942,14506,7942,8013,6403,62678,28397,28481,67026,51622,74386,21038,12756


# 5. 欠損の補完とその確認

In [8]:
data_3_4 = data_3_2.sort_values(by=['期日']) #日付によってソートする
#display(data_3_4)
data_3_5 = data_3_4.iloc[:,[0,1]] #銘柄名と期日を抽出
data_3_6 = data_3_4.iloc[:,2:].astype(float) #銘柄名と期日以外を抽出し、数値化(float)
data_3_7 = pd.concat([data_3_5,data_3_6],axis=1) #再度1つのデータフレームにする
data_3_10 = pd.DataFrame() #空のデータフレームを準備
for w in range(0,int(day_num_3)):
    data_3_8 = data_3_7.iloc[(int(com_num_3)+1)+(w-1)*(int(com_num_3))-1:((int(com_num_3)+1)+((w-1)*(int(com_num_3))))+(int(com_num_3))-1,:]
    data_3_9 = data_3_8.fillna(data_3_8.median(numeric_only=True)) #列ごとに中央値で補完(なお、文字列は対象としない)
    data_3_10 = pd.concat([data_3_10, data_3_9],axis=0) #結合
#display(data_3_10)

In [9]:
nan_num_4 = data_3_10.isnull().sum() #欠損の数を指標ごとに算出
nan_num_5 = pd.DataFrame(nan_num_4) #Seriesをデータフレーム化
nan_num_6 = nan_num_5[nan_num_5[0] != 0]  #欠損がある指標を表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(nan_num_6.T)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
#nan_num_6.index.values.tolist() #欠損がある指標を表示

""
0


# 6. 欠損の補完でも補完できなかった指標を削除

In [10]:
#不要な指標を削除
data_3_11 = data_3_10.drop(nan_num_6.index.values.tolist(), axis=1)
data_3_11 = data_3_11.sort_values(by=['銘柄コード','期日']) #銘柄コードと期日によってソートする
data_3_11

,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),PBR(普通株式数･実績･連結優先),PSR(普通株式数･日経予想･連結優先),EPS(普通株式数･日経予想･連結優先),配当利回り予想値,ﾎﾞﾘｭｰﾑﾚｼｵ,出来高回転率(250日),過去1ヶ月ﾘﾀｰﾝ,過去3ヶ月ﾘﾀｰﾝ,過去1年ﾘﾀｰﾝ,配当込み収益率(月次),HV（3ヵ月）,HV（1年）,対TOPIX β値(60ヵ月),対TOPIX α値(60ヵ月),流動比率,...,税率,総資産回転率,流動資産回転率,固定資産回転率,有形固定資産回転率,棚卸資産回転率,売上債権回転率,買入債務回転率,自己資本伸び率,総資産伸び率,売上高伸び率,営業利益伸び率,経常利益伸び率,当期利益伸び率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,3780.0,4190.0,3710.0,2.9185,-1.3674,-7.4138,60.719152,5.264058,0.252996,6.620646,0.0000,216.1662,29.704631,7.200001,0.249379,8.648647,0.070089,26.993740,34.842885,1.28069,-0.51442,103.526052,...,23.029557,2.500118,3.205759,11.358540,30.709026,10.589934,7.433289,11.914443,5.074555,-6.002066,-11.170454,-154.195804,62.164662,130.357143,0.785643,34.644268,1.057022,1.049759,1.141331,1.049765,0.946636,35.878976,4.553936e+10,4020.0,1.640167
1,極洋,1995/01,1301.0,4070.0,4130.0,3580.0,-2.8197,-3.9003,-10.0966,58.302469,5.054543,0.242927,6.620646,0.0000,203.8247,34.868735,-3.980098,-8.747042,-12.272727,-0.041676,34.855067,34.228892,1.27333,-0.44463,103.526052,...,23.029557,2.500118,3.205759,11.358540,30.709026,10.589934,7.433289,11.914443,5.074555,-6.002066,-11.170454,-154.195804,62.164662,130.357143,0.785643,34.801282,1.057022,1.049759,1.141445,1.049182,0.946636,35.878976,4.372685e+10,3860.0,1.621369
2,極洋,1995/02,1301.0,3880.0,4000.0,3460.0,-8.3524,-10.2346,-16.9429,53.167019,4.609324,0.221529,6.620646,0.0000,180.2275,37.693544,-8.808292,-6.133333,-14.146342,-0.089931,35.678951,31.980063,1.31468,-0.80345,103.526052,...,23.029557,2.500118,3.205759,11.358540,30.709026,10.589934,7.433289,11.914443,5.074555,-6.002066,-11.170454,-154.195804,62.164662,130.357143,0.785643,34.801282,1.057022,1.049759,1.141100,1.047150,0.946636,35.878976,3.987526e+10,3520.0,1.728384
3,極洋,1995/03,1301.0,3520.0,3800.0,3350.0,-3.1339,-10.3753,-17.6057,51.354507,4.452188,0.213977,6.620646,0.0000,50.4686,34.427358,-3.409090,-15.422885,-19.239904,-0.035915,44.062784,33.281984,1.25841,-0.61084,103.526052,...,23.029557,2.500118,3.205759,11.358540,30.709026,10.589934,7.433289,11.914443,5.074555,-6.002066,-11.170454,-154.195804,62.164662,130.357143,0.785643,34.424683,1.057022,1.049759,1.141879,1.049182,0.946636,35.878976,3.851588e+10,3400.0,1.583596
4,極洋,1995/04,1301.0,3300.0,3700.0,3150.0,1.7775,-3.0125,-11.4933,53.620147,4.648608,0.223417,6.620646,0.0000,70.1398,33.059091,4.411764,-8.031090,-18.577981,0.042847,43.292121,33.888829,1.25659,-0.20512,103.526052,...,23.029557,2.500118,3.205759,11.358540,30.709026,10.589934,7.433289,11.914443,5.074555,-6.002066,-11.170454,-154.195804,62.164662,130.357143,0.785643,34.424683,1.057022,1.049759,1.141879,1.049406,0.946636,35.878976,4.021511e+10,3550.0,1.579018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,883.0,900.0,773.0,-2.3335,-14.7397,-21.8114,6.358036,0.695871,0.323071,129.285201,2.3114,59.8159,52.404261,-11.422414,-24.448529,-17.841029,-0.114196,28.538277,38.864006,1.54438,0.08429,207.626921,...,39.446542,0.891352,1.883969,1.691769,2.279125,5.849084,21.035909,10.203780,10.532050,7.656144,14.754818,52.594317,62.778582,88.263391,6.271701,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,7.993457e+10,822.0,2.385110
370821,ベルーナ,2021/09,9997.0,822.0,892.0,819.0,0.2582,-4.7101,-18.3318,6.605550,0.722961,0.335648,129.285201,2.2248,148.0126,152.184196,5.065131,-11.603338,-8.694718,0.050670,25.536325,37.382646,1.54266,0.08399,207.626921,...,39.446542,0.891352,1.883969,1.691769,2.279125,5.849084,21.035909,10.203780,10.532050,7.656144,14.754818,52.594317,62.778582,88.263391,6.271701,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,8.304638e+10,854.0,2.379800
370822,ベルーナ,2021/10,999

# 7. 欠損補完後データの書き出し

In [11]:
data_3_11.to_pickle('./organized_data3.pickle') #ファイルの書き出し

In [12]:
print(os.path.getsize('./organized_data3.pickle')) #ファイルの大きさを表示

264785668


In [13]:
#先生に指標を確認してもらうためのファイルの書き出し
#with pd.ExcelWriter('/Users/nagashimahikaru/Documents/修士論文/使用データ/file for check.xlsx') as writer: 
#    pd.DataFrame(data_3_2.columns).to_excel(writer, sheet_name='分析対象候補の指標')
#    nan_num_3.to_excel(writer, sheet_name='欠損のある指標とその欠損数')
#    data_3_2.to_excel(writer, sheet_name='全データ')